In [1]:
import environmentv7 as e
from honestagentv7 import HonestAgent
import mdptoolbox
import matplotlib.pyplot as plt
import numpy as np
import progressbar as pb
import scipy.sparse as ss
import seaborn as sns
from selfishagentv7 import SelfishAgent
import warnings
warnings.filterwarnings('ignore', category=ss.SparseEfficiencyWarning)

In [44]:
# params
alpha = 0.4
gamma = 0.5
T = 8
epsilon = 10e-5

# game
action_count = 4
adopt = 0; override = 1; mine = 2; match = 3

# fork params
fork_count = 3
irrelevant = 0; relevant = 1; active = 2;

state_count = (T+1) * (T+1) * 3

# mapping utils
state_mapping = {}
states = []
count = 0
for a in range(T+1):
    for h in range(T+1):
        for fork in range(fork_count):
            state_mapping[(a, h, fork)] = count
            states.append((a, h, fork))
            count += 1

# initialize matrices
transitions = []; rewards = []
for _ in range(action_count):
    transitions.append(ss.csr_matrix(np.zeros(shape=(state_count, state_count))))
    rewards.append(ss.csr_matrix(np.zeros(shape=(state_count, state_count))))

In [49]:
def get_opt_policy(alpha, gamma, T, mining_cost):
    # populate matrices
    for state_index in range(state_count):
        a, h, fork = states[state_index]

        # adopt
        transitions[adopt][state_index, state_mapping[0, 0, irrelevant]] = 1

        # override
        if a > h:
            transitions[override][state_index, state_mapping[a-h-1, 0, irrelevant]] = 1
            rewards[override][state_index, state_mapping[a-h-1, 0, irrelevant]] = h + 1
        else:
            transitions[override][state_index, 0] = 1
            rewards[override][state_index, 0] = -10000

        # mine 
        if (fork != active) and (a < T) and (h < T):
            transitions[mine][state_index, state_mapping[a+1, h, irrelevant]] = alpha
            transitions[mine][state_index, state_mapping[a, h+1, relevant]] = (1 - alpha) 
            rewards[mine][state_index, state_mapping[a+1, h, irrelevant]] = -1 * alpha * mining_cost
            rewards[mine][state_index, state_mapping[a, h+1, relevant]] = -1 * alpha * mining_cost        
        elif (fork == active) and (a > h) and (h > 0) and (a < T) and (h < T):
            transitions[mine][state_index, state_mapping[a+1, h, active]] = alpha
            transitions[mine][state_index, state_mapping[a-h, 1, relevant]] = (1 - alpha) * gamma
            transitions[mine][state_index, state_mapping[a, h+1, relevant]] = (1 - alpha) * (1 - gamma)
            rewards[mine][state_index, state_mapping[a+1, h, active]] = -1 * alpha * mining_cost
            rewards[mine][state_index, state_mapping[a-h, 1, relevant]] = h - alpha * mining_cost
            rewards[mine][state_index, state_mapping[a, h+1, relevant]] = -1 * alpha * mining_cost
        else:
            transitions[mine][state_index, 0] = 1
            rewards[mine][state_index, 0] = -10000

        # match 
        if (fork == relevant) and (a >= h) and (h > 0) and (a < T) and (h < T):
            transitions[match][state_index, state_mapping[a+1, h, active]] = alpha
            transitions[match][state_index, state_mapping[a-h, 1, relevant]] = (1 - alpha) * gamma
            transitions[match][state_index, state_mapping[a, h+1, relevant]] = (1 - alpha) * (1 - gamma)
            rewards[match][state_index, state_mapping[a+1, h, active]] = -1 * alpha * mining_cost
            rewards[match][state_index, state_mapping[a-h, 1, relevant]] = h - alpha * mining_cost
            rewards[match][state_index, state_mapping[a, h+1, relevant]] = -1 * alpha * mining_cost
        else:
            transitions[match][state_index, 0] = 1
            rewards[match][state_index, 0] = -10000

    rvi = mdptoolbox.mdp.RelativeValueIteration(transitions, rewards, epsilon/8)
    rvi.run()
    policy = rvi.policy
    return np.reshape(policy, (9,9,3))

In [46]:
def processPolicy(policy):
    results = ''
    for a in range(9):
        for h in range(9):
            for fork in range(3):
                state_index = state_mapping[(a, h, fork)]
                action = policy[state_index]
                
                if action == 0:
                    results += 'a'
                elif action == 1:
                    results += 'o'
                elif action == 2:
                    results += 'w'
                elif action == 3:
                    results += 'm'
                else:
                    print('here')
            results += ' & '
        results += '\\\\ \n'
    print(results)

In [47]:
# params
alpha = 0.4
gamma = 0.5
T = 8
epsilon = 10e-5

In [51]:
# simulation
length = int(1e6)
mining_costs = np.arange(0, 1.1, 0.1)

bar = pb.ProgressBar()
results = []
for mining_cost in bar(mining_costs):
    optimal_policy = get_opt_policy(alpha, gamma, T, mining_cost)
    env = e.Environment(alpha, gamma, T, mining_cost)
    # simulation
    inner_bar = pb.ProgressBar()
    _ = env.reset()
    current_reward = 0
    for _ in inner_bar(range(length)):
        a, h, fork = env.current_state
        action = optimal_policy[(a, h, fork)]
        _, reward = env.takeAction(action)
        current_reward += reward
    results.append(current_reward)
    print('mining cost {:.04f}, reward per action {:.06f}'.format(mining_cost, current_reward / length))
print(rewards)

100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:40 Time:  0:00:40
  9% (1 of 11) |##                       | Elapsed Time: 0:00:40 ETA:   0:06:49

mining cost 0.0000, reward per action 0.221732


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:42 Time:  0:00:42
 18% (2 of 11) |####                     | Elapsed Time: 0:01:24 ETA:   0:06:28

mining cost 0.1000, reward per action 0.188470


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:41 Time:  0:00:41
 27% (3 of 11) |######                   | Elapsed Time: 0:02:06 ETA:   0:05:36

...

mining cost 0.8000, reward per action 0.039755


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:35 Time:  0:00:35
 90% (10 of 11) |#####################   | Elapsed Time: 0:06:53 ETA:   0:00:36

mining cost 0.9000, reward per action 0.019688


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:32 Time:  0:00:32
100% (11 of 11) |########################| Elapsed Time: 0:07:26 Time:  0:07:26


mining cost 1.0000, reward per action 0.000000
[<243x243 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>, <243x243 sparse matrix of type '<class 'numpy.float64'>'
	with 243 stored elements in Compressed Sparse Row format>, <243x243 sparse matrix of type '<class 'numpy.float64'>'
	with 413 stored elements in Compressed Sparse Row format>, <243x243 sparse matrix of type '<class 'numpy.float64'>'
	with 299 stored elements in Compressed Sparse Row format>]


In [9]:
# simulation
length = int(1e6)
mining_costs = np.arange(0, 1.1, 0.1)
honest_agent = HonestAgent()

bar = pb.ProgressBar()
rewards = []
for mining_cost in bar(mining_costs):
    env = e.Environment(alpha, gamma, T, mining_cost)
    # simulation
    inner_bar = pb.ProgressBar()
    _ = env.reset()
    current_reward = 0
    for _ in inner_bar(range(length)):
        a, h, fork = env.current_state
        action = honest_agent.act((a, h, fork))
        _, reward = env.takeAction(action)
        current_reward += reward
    rewards.append(current_reward)
    print('mining cost {:.04f}, reward per action {:.06f}'.format(mining_cost, current_reward / length))
rewards

100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:32 Time:  0:00:32
  0% (2693 of 1000000) |                 | Elapsed Time: 0:00:00 ETA:   0:00:39

mining cost 0.0000, reward per action 0.199806


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:34 Time:  0:00:34


mining cost 0.1000, reward per action 0.180655


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:35 Time:  0:00:35
  0% (5517 of 1000000) |                 | Elapsed Time: 0:00:00 ETA:   0:00:36

...

100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:36 Time:  0:00:36
  0% (2911 of 1000000) |                 | Elapsed Time: 0:00:00 ETA:   0:00:34

mining cost 0.9000, reward per action 0.020097


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:35 Time:  0:00:35
100% (11 of 11) |########################| Elapsed Time: 0:06:28 Time:  0:06:28


mining cost 1.0000, reward per action -0.000154


[199806.0,
 180654.9999999055,
 160099.0000005313,
 140665.0000009786,
 120548.99999874808,
 99993.00000089542,
 80752.99999977153,
 59634.00000038184,
 40137.000000125954,
 20096.99999984972,
 -154.0000000014394]

In [6]:
# simulation
length = int(1e6)
mining_costs = np.arange(0, 1.1, 0.1)
selfish_agent = SelfishAgent(T)

bar = pb.ProgressBar()
for mining_cost in bar(mining_costs):
    env = e.Environment(alpha, gamma, T, mining_cost)
    # simulation
    inner_bar = pb.ProgressBar()
    _ = env.reset()
    current_reward = 0
    for _ in inner_bar(range(length)):
        a, h, fork = env.current_state
        action = selfish_agent.act((a, h, fork))
        _, reward = env.takeAction(action)
        current_reward += reward
    print('mining cost {:.04f}, reward per action {:.06f}'.format(mining_cost, current_reward / length))

100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:34 Time:  0:00:34
  0% (7884 of 1000000) |                 | Elapsed Time: 0:00:00 ETA:   0:00:25

mining cost 0.0000, reward per action 0.230558


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:31 Time:  0:00:31
  0% (6033 of 1000000) |                 | Elapsed Time: 0:00:00 ETA:   0:00:33

mining cost 0.1000, reward per action 0.205044


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:28 Time:  0:00:28


...

mining cost 0.8000, reward per action 0.011908


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:29 Time:  0:00:29
  0% (6946 of 1000000) |                 | Elapsed Time: 0:00:00 ETA:   0:00:28

mining cost 0.9000, reward per action -0.016083


100% (1000000 of 1000000) |##############| Elapsed Time: 0:00:29 Time:  0:00:29
100% (11 of 11) |########################| Elapsed Time: 0:05:33 Time:  0:05:33


mining cost 1.0000, reward per action -0.044013
